# Validation tests on DC2 `calexps` and `src` catalogs

Owner: Javier Sánchez 
Date Last Run: Apr-21-2019

This notebook is intended to document part of the ongoing validation work for the DC2 simulations. For more details please check [here](https://confluence.slac.stanford.edu/display/LSSTDESC/DC2+Data+Product+Overview). The code here can be adapted for other DC2 runs.

In [ ]:
%pylab inline

Temporary fix while GCRCatalogs is updated

In [ ]:
import sys
sys.path.insert(0, '/global/homes/j/jsanch87/gcr-catalogs/')
import GCRCatalogs

In [ ]:
import GCR

In [ ]:
from GCRCatalogs.butler_interface import SingleVisitCatalog

In [ ]:
import lsst.daf.persistence
from scipy.stats import binned_statistic

In [ ]:
import matplotlib
matplotlib.rcParams.update({'font.size': 14})

We are going to read some calexps and the `OpSim` database to select interesting visits

In [ ]:
data_imsim = '/global/cscratch1/sd/desc/DC2/data/Run1.2i_globus_in2p3_20181217/w_2018_39/rerun/281118'

In [ ]:
data_phosim = '/global/cscratch1/sd/desc/DC2/data/Run1.2p_v4/w_2018_39/rerun/calexp-v4'

In [ ]:
db_file = '/global/projecta/projectdirs/lsst/groups/SSim/DC2/minion_1016_desc_dithered_v4.db'

In [ ]:
import sqlite3
from sqlite3 import Error
 
def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return None

In [ ]:
conn = create_connection(db_file)

We are going to check the visits that are close to the zenith and with seeing ~ 0.7"

In [ ]:
def get_visits(conn,min_alt=80,min_seeing=0.60,max_seeing=0.8):
    cur = conn.cursor()
    cur.execute("SELECT obsHistID, filter FROM ObsHistory WHERE altitude>%f and finSeeing >%f and finSeeing < %f and filter='r'" % (np.radians(min_alt),min_seeing,max_seeing))
    return cur.fetchall()

We need the list of available visits to cross-check with the database

In [ ]:
butler = lsst.daf.persistence.Butler(data_imsim)

In [ ]:
datarefs = butler.subset('calexp') # This gives us a list with all `calexp`s

Let's query the visits that have altitude larger than 80 degrees and seeing ~0.7"

In [ ]:
good_visits = np.array(get_visits(conn))[:,0].astype(int)

In [ ]:
len(good_visits)

So we have 4750 candidates, let's see how many were simulated

In [ ]:
visitno = []
for visitId in datarefs.cache:
    aux = visitId['visit']
    if visitId['visit'] in good_visits:
        #print(visitId)
        visitno.append(aux)
visitno=np.unique(visitno)

In [ ]:
visits_to_check = visitno[np.in1d(visitno, good_visits)]

In [ ]:
len(visits_to_check)

Only 22 of them were simulated. We are going to query some of them using the `SingleVisitCatalog` class as in `DC2_calexp_src_validation.ipynb` tutorial. We will select only a few columns.

In [ ]:
columns = ['base_ClassificationExtendedness_value', # This is a basic star/galaxy separation flag more on Bosch et al 2018
           'ext_shapeHSM_HsmShapeRegauss_e1', # e1 using GalSim's HSM
           'ext_shapeHSM_HsmShapeRegauss_e2', # e2 using GalSim's HSM
           'base_SdssShape_xx', # xx moment using SDSS algorithm
           'base_SdssShape_xy', # xy moment using SDSS algorithm
           'base_SdssShape_yy', # yy moment using SDSS algorithm
           'base_SdssShape_psf_xx', # xx moment of the PSF in the position of this object using SDSS algorithm
           'base_SdssShape_psf_xy', # xy as above
           'base_SdssShape_psf_yy', # yy as above
           'base_PsfFlux_instFlux', # PSF-flux
           'base_PsfFlux_instFluxErr', # PSF-flux error
           'base_PsfFlux_mag', # magnitude computed in a PSF magnitude (derived from base_PsfFlux_instFlux with the corresponding zeropoint)
           'base_PsfFlux_magErr', # corresponding error to the above magnitude
]

In [ ]:
nvisits_to_query = 3 # We are going to check 3 visits only but feel free to query up to 22!
for i, vv in enumerate(visits_to_check[:nvisits_to_query]):
    gc_data = SingleVisitCatalog(repo=data_imsim, filter_band='r', visit=vv, detector=None)
    if i==0:
        data_im = gc_data.get_quantities(columns, filters=['deblend_nChild == 0'])
    else:
        aux = gc_data.get_quantities(columns, filters=['deblend_nChild == 0'])
        for key in data_im.keys():
            data_im[key] = np.concatenate((data_im[key], aux[key])).ravel()

In [ ]:
# We define some routines to convert to the LSST-SRD convention for |e|
def asymQ(ixx,iyy,ixy):
    asymQx = ixx - iyy
    asymQy = 2*ixy
    return np.sqrt(asymQx**2 + asymQy**2)
def trQ(ixx,iyy):
    return ixx+iyy
def get_a(ixx,iyy,ixy):
    return np.sqrt(0.5*(trQ(ixx,iyy)+asymQ(ixx,iyy,ixy)))
def get_b(ixx,iyy,ixy):
    return np.sqrt(0.5*(trQ(ixx,iyy)-asymQ(ixx,iyy,ixy)))
def get_e(ixx,iyy,ixy):
    a = get_a(ixx,iyy,ixy)
    b = get_b(ixx,iyy,ixy)
    return (a**2-b**2)/(a**2+b**2)

So let's compute the PSF ellipticity in our 3 visits

In [ ]:
e_psf_i = get_e(data_im['base_SdssShape_psf_xx'],
                data_im['base_SdssShape_psf_yy'],
                data_im['base_SdssShape_psf_xy'])

Show the distribution, the median and 95-th percentile. There are certain criteria for these quantities described in the LSST Science Requirement Document: [LPM-17](https://docushare.lsst.org/docushare/dsweb/Get/LPM-17)

In [ ]:
plt.hist(np.abs(e_psf_i),histtype='step',range=(0,0.2),bins=100,normed=True)
plt.plot(np.median(e_psf_i)*np.ones(3),np.linspace(0,70,3),'k--')
plt.plot(np.percentile(e_psf_i,95)*np.ones(3),np.linspace(0,70,3),'r--')
plt.xlabel(r'$|e|=(1-q^{2})/(1+q^{2})$',fontsize=16)
plt.ylabel(r'$P|e|$',fontsize=16)
#plt.legend(loc='best')
plt.xlim(0,0.07)

The black line represents the median of the distribution (that should be below 0.04) and the red line represents the 95th percentile (that should be below 0.07). Both criteria are fulfilled! In fact, we increased the ellipticity for run 2.1i to make it more realistic!